In [40]:
!python -V

Python 3.9.12


In [41]:
import pandas as pd

In [46]:
import seaborn as sns
import matplotlib.pyplot as plt

In [75]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [55]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df["duration"]=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[((df.duration >= 1) & (df.duration <=60))]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df
    

In [66]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')


In [67]:
categorical = ["PU_DO"]
numerical = ['trip_distance']

In [77]:
df_train['PU_DO']=df_train["PULocationID"]+'_'+df_train["DOLocationID"]
df_val['PU_DO']=df_val["PULocationID"]+'_'+ df_val["DOLocationID"]

In [69]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical +numerical].to_dict(orient="records")

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [70]:
target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

In [71]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.479562160810692

In [74]:
ls = Lasso(alpha = 0.0001)
ls.fit(X_train, y_train)

y_pred = ls.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.616617761096093

In [76]:
with open("models/linreg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)